# From Modeling to Evaluation


## Introduction

In this lab, we will continue learning about the data science methodology, and focus on the **Modeling** and **Evaluation** stages.

## Objectives

After complting this lab you will be able to:

* Create Models
* Evaluate the models


## Table of Contents


<div class="alert alert-block alert-info" style="margin-top: 20px">

1. [Recap](#0)<br>
2. [Data Modeling](#2)<br>
3. [Model Evaluation](#4)<br>
</div>
<hr>


# Recap <a id="0"></a>

In Lab **From Understanding to Preparation**, we explored the data and prepared it for modeling.


The data was compiled by a researcher named Yong-Yeol Ahn, who scraped tens of thousands of food recipes (cuisines and ingredients) from three different websites, namely:


<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0103EN-SkillsNetwork/labs/Module%202/images/lab2_fig3_allrecipes.png" width="500">
<div align="center">
www.allrecipes.com
</div>
<br/><br/>
<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0103EN-SkillsNetwork/labs/Module%202/images/lab2_fig4_epicurious.png" width="500">
<div align="center">
www.epicurious.com
</div>
<br/><br/>
<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0103EN-SkillsNetwork/labs/Module%202/images/lab2_fig5_menupan.png" width="500">
<div align="center">
www.menupan.com
</div>
<br/><br/>


For more information on Yong-Yeol Ahn and his research, you can read his paper on [Flavor Network and the Principles of Food Pairing](http://yongyeol.com/papers/ahn-flavornet-2011.pdf).


We will repeat the preprocessing steps that we implemented in Lab **From Understanding to Preparation** in order to prepare the data for modeling. For more details on preparing the data, please refer to Lab **From Understanding to Preparation**.


In [1]:
import pandas as pd

recipes = pd.read_csv('recipes.csv')

<hr>


# Data Modeling <a id="2"></a>


<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0103EN-SkillsNetwork/labs/Module%204/images/flowchart_data_modeling.png" width="500">


Let's start by importing and installing the R libraries relevant to decision trees


In [ ]:
# install libraries if not installed
if("rpart" %in% rownames(installed.packages()) == FALSE) {install.packages("rpart", 
    repo = "http://mirror.las.iastate.edu/CRAN/")}
if("rpart.plot" %in% rownames(installed.packages()) == FALSE) {install.packages("rpart.plot", 
    repo = "http://mirror.las.iastate.edu/CRAN/")}

# load libraries
library(rpart)
library(rpart.plot)

print("Libraries loaded!")

Check the data again!


In [ ]:
head(recipes)

## [bamboo_tree] Only Asian and Indian Cuisines

Here, we are creating a decision tree for the recipes for just some of the Asian (Korean, Japanese, Chinese, Thai) and Indian cuisines. The reason for this is because the decision tree does not run well when the data is biased towards one cuisine, in this case American cuisines. One option is to exclude the American cuisines from our analysis or just build decision trees for different subsets of the data. Let's go with the latter solution.


Let's build our decision tree using the data pertaining to the Asian and Indian cuisines and name our decision tree *bamboo_tree*.


In [ ]:
# select subset of cuisines
bamboo_tree <- rpart(formula=cuisine ~ ., data=recipes[recipes$cuisine %in% c("korean", 
                                                                              "japanese", 
                                                                              "chinese", 
                                                                              "thai",
                                                                              "indian"),], method ="class")

print("Decision tree model saved to bamboo_tree!")

Let's plot the decision tree and examine how it looks like.


In [ ]:
# plot bamboo_tree
rpart.plot(bamboo_tree, type=3, extra=2, under=TRUE, cex=0.75, varlen=0, faclen=0)

The decision tree learned, using the right most branch of the tree,:
* If a recipe contains *cumin* and *fish*, then it is most likely a **Thai** recipe.
* If a recipe contains *cumin* but no *fish*, then it is most likely an **Indian** recipe.
* If a recipe does not contain *cumin* and contains *roasted_sesame_seed*, then it is most likely a **Korean** recipe.


You can analyze the remaining branches of the tree to come up with similar rules for determining the cuisine of different recipes. 


Feel free to select another subset of cuisines and build a decision tree of their recipes. You can select some European cuisines and build a decision tree to explore the ingredients that differentiate them.


# Model Evaluation <a id="4"></a>


<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0103EN-SkillsNetwork/labs/Module%204/images/flowchart_evaluation.png" width="500">


To evaluate our model of Asian and Indian cuisines, we will split our dataset into a training set and a test set. We will build the decision tree using the training set. Then, we will test the model on the test set and compare the cuisines that the model predicts to the actual cuisines. 


Let's first create a new dataframe using only the data pertaining to the Asian and Indian cuisines, and let's call the new dataframe **bamboo**.


In [ ]:
bamboo <- recipes[recipes$cuisine %in% c("korean", "japanese", "chinese", "thai", "indian"),]

Let's see how many recipes exist for each cuisine.


In [ ]:
base::table(as.factor(as.character(bamboo$cuisine)))

Let's remove 30 recipes from each cuisine to use as the test set, and let's name this set **bamboo_test**.


In [ ]:
# set sample size
sample_n <- 30

Create a dataframe containing 30 recipes from each cuisine, selected randomly.


In [ ]:
# take 30 recipes from each cuisine
set.seed(4) # set random seed
korean <- bamboo[base::sample(which(bamboo$cuisine == "korean") , sample_n), ]
japanese <- bamboo[base::sample(which(bamboo$cuisine == "japanese") , sample_n), ]
chinese <- bamboo[base::sample(which(bamboo$cuisine == "chinese") , sample_n), ]
thai <- bamboo[base::sample(which(bamboo$cuisine == "thai") , sample_n), ]
indian <- bamboo[base::sample(which(bamboo$cuisine == "indian") , sample_n), ]

# create the dataframe
bamboo_test <- rbind(korean, japanese, chinese, thai, indian)

Check that there are 30 recipes for each cuisine.


In [ ]:
# check that we have 30 recipes from each cuisine
base::table(as.factor(as.character(bamboo_test$cuisine)))

Next, let's create the training set by removing the test set from the **bamboo** dataset, and let's call this set **bamboo_train**.


In [ ]:
bamboo_train <- bamboo[!(rownames(bamboo) %in% rownames(bamboo_test)),]

Check that there are 30 _fewer_ recipes now for each cuisine.


In [ ]:
base::table(as.factor(as.character(bamboo_train$cuisine)))

Let's build the decision tree using the training set, **bamboo_train**, and name the generated tree **bamboo_tree_pred** for prediction.


In [ ]:
bamboo_train_tree <- rpart(formula=cuisine ~ ., data=bamboo_train, method="class")

Let's plot the decision tree and explore it.


In [ ]:
rpart.plot(bamboo_train_tree, type=3, extra=2, under=TRUE, cex=0.75, varlen=0, faclen=0)

It is obvious how removing 30 recipes from each cuisine resulted in more decisions in the tree.


#### Now let's test our model on the test data.


In [ ]:
bamboo_pred_cuisines <- predict(bamboo_train_tree, subset(bamboo_test, select=-c(cuisine)), type="class")

To quantify how well the decision tree is able to determine the cuisine of each recipe correctly, we will create a confusion matrix which presents a nice summary on how many recipes from each cuisine are correctly classified. It also sheds some light on what cuisines are being confused with what other cuisines.


So let's go ahead and create the confusion matrix for how well the decision tree is able to correctly classify the recipes in **bamboo_test**.


In [ ]:
bamboo_confusion_matrix <- base::table(
    paste(as.character(bamboo_test$cuisine),"_true", sep=""),
    paste(as.character(bamboo_pred_cuisines),"_pred", sep="")
)

round(prop.table(bamboo_confusion_matrix, 1)*100, 1)

The rows represent the actual cuisines from the dataset and the columns represent the predicted ones. Each row should sum to 100%. We make the following observations:

* Using the first row in the confusion matrix, 60% of the **Chinese** recipes in **bamboo_test** were correctly classified by our decision tree whereas 36.7% of the **Chinese** recipes were misclassified as **Korean** and 3.3% were misclassified as **Japenese**.

* Using the Indian row, 90% of the **Indian** recipes in **bamboo_test** were correctly classified by our decision tree and 10% of the **Indian** recipes were misclassified as **Korean**.


How many **Japanese** recipes were correctly classified by our decision tree?


<div align="right">
<a href="#p1" class="btn btn-default" data-toggle="collapse">Click here for the solution</a>
</div>
<div id="p1" class="collapse">
33.3%.
</div>


How many **Korean** recipes were misclassified as **Japanese**?


<div align="right">
<a href="#p2" class="btn btn-default" data-toggle="collapse">Click here for the solution</a>
</div>
<div id="p2" class="collapse">
16.7%.
</div>


What cuisine has the least number of recipes correctly classified by the decision tree?


<div align="right">
<a href="#p3" class="btn btn-default" data-toggle="collapse">Click here for the solution</a>
</div>
<div id="p3" class="collapse">
<strong>Japanese</strong> cuisine, with 33.3% only.
</div>


<br>
<hr>
